In [ ]:
class Contract:
    def loadContract(self):
        print('Loading contract from {}...'.format(self.compiled_contract_location))
        with open(self.compiled_contract_location, "r") as file:
            contractInfo = json.load(file)
        # Get bytecode
        self.bytecode = contractInfo["contracts"]["HashStorage.sol"]["HashStorage"]["evm"]["bytecode"]["object"]
        # get abi
        self.abi = contractInfo["contracts"]["HashStorage.sol"]["HashStorage"]["abi"]
        # get contract address
        load_dotenv(find_dotenv())
        self.contract_address = os.getenv('contract_address')
        print(os.getenv('contract_address'))
        self.contract = self.w3.eth.contract(address=self.contract_address, abi=self.abi) # our contract
        print('Loaded!')
    
    def interactDemo(self):
        nonce = self.w3.eth.getTransactionCount(
                    self.my_address
                )  # gives our nonce - number of transactions
        # initial value of hashNumber:
        print(
            "Current value of favorite number: "
            + str(self.contract.functions.retrieve().call())
        )  # No state change!

        print("Updating contract...")
        # Store new value for hashNumber:
        store_transaction = self.contract.functions.store("0x8888").build_transaction(
            {
                "gasPrice": self.w3.eth.gas_price,
                "chainId": self.chain_id,
                "from": self.my_address,
                "nonce": nonce,
            }
        )
        signed_store_tx = self.w3.eth.account.sign_transaction(
            store_transaction, private_key=self.private_key
        )
        send_store_tx = self.w3.eth.send_raw_transaction(signed_store_tx.rawTransaction)
        tx_receipt = self.w3.eth.wait_for_transaction_receipt(send_store_tx)
        print("Updated!")
        print(
            "New value of hash: " + str(self.contract.functions.retrieve().call())
        )
        print(self.contract_address)